In [2]:
# Import libraries and Packages
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime as dt
from datetime import datetime,tzinfo
from pytz import timezone
import time
import pytz
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pprint as pp


## Filter news by marker
This notebook is intended to perform the following processes:

    1.1 Creates marks for mapping statistically significant price changes to news articles within a window of time

___

In [3]:
marker= pd.read_csv('markerPosi.csv')
marker.head()
mark=marker['Date'] #create a new column of dates to track events
mark.head(5)

0    2018-02-04
1    2018-02-06
2    2018-02-07
3    2018-02-08
4    2018-02-09
Name: Date, dtype: object

__Modify datestamp for mapping purposes__

In [4]:
# create list of timestamps according to given datetime parameters
array_mark = []
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date)
pp.pprint(array_mark[0:5])

[datetime.datetime(2018, 2, 4, 0, 0),
 datetime.datetime(2018, 2, 6, 0, 0),
 datetime.datetime(2018, 2, 7, 0, 0),
 datetime.datetime(2018, 2, 8, 0, 0),
 datetime.datetime(2018, 2, 9, 0, 0)]


__Allows for a given date range of consideration (+- N days from statistically significant price change)__

In [5]:
N = 3
days_before=[]
days_after=[]
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    days_before.append((date-timedelta(days=N)).isoformat())
    days_after.append((date+timedelta(days=N)).isoformat())

In [6]:
# appends additional days to list -- used for mapping of articles
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date-timedelta(days=1))
    array_mark.append(date+timedelta(days=1))
    array_mark.append(date-timedelta(days=2))
    array_mark.append(date+timedelta(days=2))
    array_mark.append(date-timedelta(days=3))
    array_mark.append(date+timedelta(days=3))
pp.pprint(array_mark[:10])

[datetime.datetime(2018, 2, 4, 0, 0),
 datetime.datetime(2018, 2, 6, 0, 0),
 datetime.datetime(2018, 2, 7, 0, 0),
 datetime.datetime(2018, 2, 8, 0, 0),
 datetime.datetime(2018, 2, 9, 0, 0),
 datetime.datetime(2018, 2, 10, 0, 0),
 datetime.datetime(2018, 2, 11, 0, 0),
 datetime.datetime(2018, 2, 12, 0, 0),
 datetime.datetime(2018, 2, 13, 0, 0),
 datetime.datetime(2018, 2, 14, 0, 0)]


In [7]:
# 1x8 array of 1's
np.ones(8)

array([1., 1., 1., 1., 1., 1., 1., 1.])

__Mark 'significant' dates__

In [8]:
# Marking of dates with statistically significant price changes
date_mark=pd.DataFrame({'Date':array_mark,
                       'Mark':np.ones(len(array_mark))})
pp.pprint(date_mark.head())
date_mark.to_csv('positive_marked_dates.csv')

        Date  Mark
0 2018-02-04   1.0
1 2018-02-06   1.0
2 2018-02-07   1.0
3 2018-02-08   1.0
4 2018-02-09   1.0


In [46]:
## checking variable types ##
#type(date_mark['Date'][0])  # .Timestamp
#type(date_mark['Mark'][0]) # float type

__Prepare articles for marking__

In [12]:
df= pd.read_csv('article_data_and_price_labeled_publisher.csv').drop('Unnamed: 0', axis = 1)
df

,author,contents,description,publisher,source_url,timeStamp,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,959
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,365
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1181
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,241
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,452
5,Financial Times,hannah murphy london bitflyer tokyo based oper...,Exchange to offer trading of bitcoin/euro pair...,Financial Times,https://www.ft.com/content/5bf1462c-ff84-11e7-...,2018-01-23 00:06:21+00:00,Japan’s bitFlyer set to stage Europe expansion,2018-01-23,00:06:21,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,364
6,Topix.com,not topix user yet forgot your password news p...,The fundamentals are as untenable as they were...,Topix.com,http://www.topix.com/tech/p2p/2018/01/dont-try...,2018-01-23 00:11:59+00:00,Don't Try To Catch The Bitcoin Knife,2018-01-23,00:11:59,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1085
7,Chloe Aiello,var postloadfunctions var foresee enabled var ...,ETF Managers Group Founder and CEO Sam Masucci...,CNBC,https://www.cnbc.com/2018/01/22/marijuana-etf-...,2018-01-23 00:26:00+00:00,Here's why one investor is way more comfortabl...,2018-01-23,00:26:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,146
8,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,Miami has apparently witnessed the “hottest” w...,Bitcoinist.com,http://bitcoinist.com/achain-showcased-collabo...,2018-01-23 00:30:10+00:00,Achain Showcased its Collaborative Platform at...,2018-01-23,00:30:10,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,96
9,Sasha Lekach,the bitcoin craze officially jumped real estat...,The bitcoin craze has officially jumped to rea...,Mashable,http://mashable.com/2018/01/22/cryptohomes-rea...,2018-01-23 

In [17]:
#Convert Timestamp into separate Date and Time
#df['Date'] = pd.to_datetime(df['timeStamp']).dt.date

del df['timeStamp'] #Delete original datetime column
df.head()
print(date_mark)

KeyError: 'timeStamp'

__Additional Pre-Processing__

In [18]:
# datatype coercion of features
df.date = df.date.astype(str)               
date_mark.Date = date_mark.Date.astype(str)

__Mark articles according to statistically significant price changes__

In [23]:
marked = pd.merge(df,date_mark, how='left', left_on='date',right_on='Date')

In [24]:
# handle missing marks
marked.Mark = marked.Mark.fillna(0)

In [28]:
marked[marked['Mark']==1]

,author,contents,description,publisher,source_url,title,date,time,Open,High,...,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date,Mark
2856,Andy Spence,the rsa uses cookies website by using website ...,Despite high-profile issues with digital work ...,Thersa.org,https://www.thersa.org/discover/publications-a...,Blockchain platforms can enable good work,2018-02-01,00:00:00,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,1050,2018-02-01,1.0
2857,Tabassum Naiz,real time prices vires numeris bitcoin ethereu...,Tokyo-based SBI Holdings has announced that it...,Bitcoinist.com,http://bitcoinist.com/sbi-holdings-to-list-xrp...,SBI Holdings to Exclusively List Ripple’s XRP ...,2018-02-01,00:00:37,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,96,2018-02-01,1.0
2858,NATHANIEL POPPER,advertisement by nathaniel popperjan a growing...,Concerns are growing that activity on the virt...,The New York Times,https://www.nytimes.com/2018/01/31/technology/...,Worries Grow That the Price of Bitcoin Is Bein...,2018-02-01,00:02:42,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,1000,2018-02-01,1.0
2859,"Amanda Lee, Amanda Lee",bitcoin growing popularity icos raised concern...,China’s major social media networks and search...,Scmp.com,http://www.scmp.com/tech/china-tech/article/21...,Bitcoin ads disappear from social media in Chi...,2018-02-01,00:04:45,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,883,2018-02-01,1.0
2860,Phil Glazer,in many ways title cryptocurrency misleading c...,"In many ways, the title “cryptocurrency” can b...",Hackernoon.com,https://hackernoon.com/the-different-categorie...,The Different Categories of Cryptocurrencies,2018-02-01,00:05:35,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,452,2018-02-01,1.0
2861,Bizop Team,some say cryptocurrency phenomenon wave future...,Some say the cryptocurrency phenomenon is the ...,Business-opportunities.biz,http://www.business-opportunities.biz/2018/01/...,Surf the Cryptocurrency Wave with Your Own Cry...,2018-02-01,00:32:45,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,133,2018-02-01,1.0
2862,Scott Scanlon,cnet tambi n est disponible en espa ol the pho...,CNET también está disponible en español. The p...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/sams...,"Samsung meets Bitcoin? Yep, the firm is making...",2018-02-01,01:03:12,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,1181,2018-02-01,1.0
2863,Marco Lancaster,NaN,After contemplating the brilliance of the hard...,Gizchina.com,https://www.gizchina.com/2018/01/31/samsung-no...,Samsung now manufacturing Bitcoin Mining chipsets,2018-02-01,01:03:52,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,420,2018-02-01,1.0
2864,CoinTelegraph By Darryn Pollock,cointelegraph just like regulators starting ta...,Taxing cryptocurrency gains have always been a...,Cointelegraph.com,https://cointelegraph.com/news/the-tax-man-is-...,The Tax Man is Finally Catching Up On the Cryp...,2018-02-01,01:06:36,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531,199,2018-02-01,1.0
2865,Nathan Sexer,variabl cco nathan sexer undertakes exhaustive...,"CCO @VariabL (@Consensys)—­­ Blockchain, Ether...",Consensys.net,https://media.consensys.net/state-of-decentral...,"State of Decentralized Exchanges, 2018 – Conse...",2018-02-01,01:16:44,9497.854597,9507.023528,...,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,41

In [29]:
marked.to_csv('1027_positive_marked_news.csv')

___